## Garbage Classifier 🗑️ 🚮 🚯

### Setup ⚙️

In [116]:
# Import libraries
import os
from dotenv import load_dotenv

In [117]:
# Reload for caching
from importlib import reload
import src.dataset as dataset
reload(dataset)
import src.data_loader as data_loader
reload(data_loader)

from src.dataset import BaseDataset
from src.data_loader import list_data_and_prepare_labels, split_data, split_data_to_dicts

In [118]:
# get file directory
curr_dir = os.getcwd()

# load dataset_path
dotenv_path = os.path.join(curr_dir, ".env")
load_dotenv(dotenv_path)

True

In [119]:
# Constants
DEFAULT_EPOCHS = 12
TEST_SPLIT = 0.2
VAL_SPLIT = 0.2
BATCH_SIZE = 64

In [120]:
# Variables
dataset_path = os.getenv("DATASET_LOCAL_PATH")
normalized_path = dataset_path
normalized_path

'D:/chris/Documents/UofC/MEng Soft/winter/ENEL 645/ENEL 645/ENEL 645/A2/small_dataset'

### Data Loader ↻

In [121]:
images_path = normalized_path + "/**/*.png"
images, labels_int, classes = list_data_and_prepare_labels(images_path)

In [123]:
train_images, train_labels, val_images, val_labels, test_images, test_labels = split_data(images, labels_int, VAL_SPLIT, TEST_SPLIT)